<a href="https://colab.research.google.com/github/ranjini-rao/Neural-Network_PharmaProbe/blob/main/BERT_Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Load the CSV file into a DataFrame
review_df = pd.read_csv('medical_data.csv')

# Display the first few rows of the DataFrame to verify it was loaded correctly
review_df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,lengthReview,conditionCluster_label,drugNameCluster_label
0,95260,Guanfacine,ADHD,My son is halfway through his fourth week of I...,8,27-Apr-10,192,712,2,4
1,92703,Lybrel,Birth Control,I used to take another oral contraceptive whic...,5,14-Dec-09,17,708,9,6
2,138000,Ortho Evra,Birth Control,This is my first time using any form of birth ...,8,3-Nov-15,10,428,9,4
3,35696,Buprenorphine naloxone,Opiate Dependence,Suboxone has completely turned my life around ...,9,27-Nov-16,37,669,0,2
4,155963,Cialis,Benign Prostatic Hyperplasia,2nd day on 5mg started to work with rock hard ...,2,28-Nov-15,43,373,0,5


In [3]:
#Because Google Colab is crashing I am going to only continue with the review column
#And the unique ID so I can merge the embeddings back into the df:

review_df = review_df[['uniqueID', 'review']]

In [4]:
#check the dataframe

review_df.head()

,uniqueID,review
0,95260,My son is halfway through his fourth week of I...
1,92703,I used to take another oral contraceptive whic...
2,138000,This is my first time using any form of birth ...
3,35696,Suboxone has completely turned my life around ...
4,155963,2nd day on 5mg started to work with rock hard ...


In [5]:
#1 Pre-process the data - Good
#2 Load the data - Good
#3 Tokenize the Data - Word Tokenization - break the sentences down to each word
   #A. Remove Stopwords
   #B. Lowercase
   #C. Remove punctuation
#4 Padding? - Make them the same length?
#5 Embedding - BERT
#6 Normilization - Maybe Reduce Dimensions - PCA is one way - Try each way

In [6]:
#Remove stopwords - received code from chatgpt

import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords (run this line once if you haven't downloaded stopwords before)
nltk.download('stopwords')

# Get list of English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from text
def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Remove stopwords from the 'review' column
review_df['review'] = review_df['review'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
#this step was not filtering out as many words as I would like, I found another method to remove stopwords
#going to use both methods

In [8]:
from nltk.tokenize import word_tokenize

# Download NLTK resources (run this line once if you haven't downloaded resources before)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Get list of English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords and non-content words based on POS
def remove_non_content_words(text):
    # Tokenize text into words
    tokens = word_tokenize(text)
    # Tag words with their POS
    tagged_words = nltk.pos_tag(tokens)
    # Filter out non-content words based on POS
    filtered_tokens = [word for word, pos in tagged_words if pos.startswith('NN') or pos.startswith('VB')]
    return ' '.join(filtered_tokens)

# Remove stopwords and non-content words from the 'review' column
review_df['review'] = review_df['review'].apply(remove_non_content_words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
#lowercase

# Convert text in the 'review' column to lowercase using .loc
review_df.loc[:, 'review'] = review_df['review'].str.lower()

In [11]:
#Vinaya - Replace with PUNCT
import string

# Function to remove punctuation from text
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Remove punctuation from the 'review' column using .loc
review_df.loc[:, 'review'] = review_df['review'].apply(remove_punctuation)

In [12]:
review_df.head()

,uniqueID,review
0,95260,son week intuniv became began week started tak...
1,92703,used take contraceptive pill cycle light perio...
2,138000,time using birth control im glad went months d...
3,35696,suboxone turned life feel healthier im excelli...
4,155963,day started work rock erections experianced he...


In [13]:
from transformers import BertTokenizer, BertModel
import torch

In [14]:
# Load pretrained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [15]:
# Tokenize text data - tokenize each sentence/document
tokenized_texts = [tokenizer.encode(text, add_special_tokens=True) for text in review_df['review']]

In [16]:
len(tokenized_texts)

95911

In [17]:
#Previously I was trying to embed each word, that was not working for a number of reasons, commenting out this code.

#from torch.nn.utils.rnn import pad_sequence

# Convert tokenized sequences to PyTorch tensors and pad them to the same length
# padded_sequences = pad_sequence([torch.tensor(text) for text in tokenized_texts], batch_first=True)

# padded_sequences now contains the tokenized sequences padded to the same length

# Generate attention mask
# attention_mask = (padded_sequences != 0)  # Create a mask where 0 indicates padding tokens

# Embed with the BERT model we loaded
# Pass padded_sequences through the embedding model

#with torch.no_grad():
#    embeddings = model(input_ids=padded_sequences, attention_mask=attention_mask)


In [18]:
# Commenting out padding becuase it was part of the embedding each word strategy

#padded_sequences.shape

In [19]:
# Commenting out attention_mask becuase it was part of the embedding each word strategy

#attention_mask.shape

In [20]:
import numpy as np

# Define batch size
batch_size = 20

# Initialize list to store embeddings
bert_embeddings = []

# Set the model to evaluation mode
model.eval()

# Process tokenized texts in batches
for i in range(0, len(tokenized_texts), batch_size):
    batch_tokenized_texts = tokenized_texts[i:i+batch_size]
    batch_embeddings = []
    for j, tokens in enumerate(batch_tokenized_texts):
        # Convert tokens to tensor
        tokens_tensor = torch.tensor(tokens).unsqueeze(0)

        # Forward pass, get hidden states
        with torch.no_grad():
            outputs = model(tokens_tensor)

        # Extract embeddings from the second-to-last layer (CLS token)
        embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

        # Append embeddings to the batch embeddings
        batch_embeddings.append(embeddings)

    # Extend the list of embeddings with batch embeddings
    bert_embeddings.extend(batch_embeddings)

    # Print a message when the batch is finished
    print(f"Batch {i//batch_size + 1} processed.")

# Convert the list of embeddings to a numpy array
bert_embeddings_array = np.array(bert_embeddings)



In [22]:
len(bert_embeddings_array)

95911

In [23]:
# Convert BERT embeddings array to a DataFrame
bert_embeddings_df = pd.DataFrame(bert_embeddings_array)

In [24]:
bert_embeddings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.010977,0.010914,0.200967,-0.229490,-0.535286,0.012419,0.578620,0.578304,-0.176901,-0.332041,...,-0.416409,-0.364040,-0.036060,0.383963,0.176255,-0.147201,-0.243359,-0.541467,0.062160,0.049585
1,0.066320,0.189584,0.369006,-0.046920,-0.473988,-0.238288,0.341089,0.252703,0.104143,0.059248,...,-0.279307,-0.419729,-0.389261,0.328398,0.291834,-0.027217,-0.359790,-0.706709,0.047264,-0.017902
2,0.084101,-0.019134,0.294494,0.029783,-0.228783,0.170102,0.185404,0.210424,-0.081424,-0.105462,...,-0.232733,-0.031823,-0.032784,0.188440,0.162272,0.363399,-0.096650,-0.693634,-0.024901,0.548486
3,0.007820,0.207558,0.179105,-0.210057,-0.197015,0.104799,0.338058,0.038666,0.026772,-0.294704,...,-0.310564,-0.599643,-0.375174,0.309915,0.577983,0.051811,-0.184821,-0.710691,0.065533,0.371945
4,-0.193177,0.360585,0.448292,-0.253824,-0.532782,0.085381,0.607802,0.260307,-0.136089,-0.218225,...,-0.247983,-0.438636,-0.037911,-0.030183,0.508780,0.064493,-0.205261,-0.527391,-0.101341,0.039573


In [25]:
# Add the uniqueID column to bert_embeddings_df
bert_embeddings_df['uniqueID'] = review_df['uniqueID'].tolist()

In [26]:
bert_embeddings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,uniqueID
0,-0.010977,0.010914,0.200967,-0.229490,-0.535286,0.012419,0.578620,0.578304,-0.176901,-0.332041,...,-0.364040,-0.036060,0.383963,0.176255,-0.147201,-0.243359,-0.541467,0.062160,0.049585,95260
1,0.066320,0.189584,0.369006,-0.046920,-0.473988,-0.238288,0.341089,0.252703,0.104143,0.059248,...,-0.419729,-0.389261,0.328398,0.291834,-0.027217,-0.359790,-0.706709,0.047264,-0.017902,92703
2,0.084101,-0.019134,0.294494,0.029783,-0.228783,0.170102,0.185404,0.210424,-0.081424,-0.105462,...,-0.031823,-0.032784,0.188440,0.162272,0.363399,-0.096650,-0.693634,-0.024901,0.548486,138000
3,0.007820,0.207558,0.179105,-0.210057,-0.197015,0.104799,0.338058,0.038666,0.026772,-0.294704,...,-0.599643,-0.375174,0.309915,0.577983,0.051811,-0.184821,-0.710691,0.065533,0.371945,35696
4,-0.193177,0.360585,0.448292,-0.253824,-0.532782,0.085381,0.607802,0.260307,-0.136089,-0.218225,...,-0.438636,-0.037911,-0.030183,0.508780,0.064493,-0.205261,-0.527391,-0.101341,0.039573,155963


In [27]:
import pandas as pd

# Load the CSV file into a DataFrame
medical_df = pd.read_csv('medical_data.csv')

# Display the first few rows of the DataFrame to verify it was loaded correctly
medical_df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,lengthReview,conditionCluster_label,drugNameCluster_label
0,95260,Guanfacine,ADHD,My son is halfway through his fourth week of I...,8,27-Apr-10,192,712,2,4
1,92703,Lybrel,Birth Control,I used to take another oral contraceptive whic...,5,14-Dec-09,17,708,9,6
2,138000,Ortho Evra,Birth Control,This is my first time using any form of birth ...,8,3-Nov-15,10,428,9,4
3,35696,Buprenorphine naloxone,Opiate Dependence,Suboxone has completely turned my life around ...,9,27-Nov-16,37,669,0,2
4,155963,Cialis,Benign Prostatic Hyperplasia,2nd day on 5mg started to work with rock hard ...,2,28-Nov-15,43,373,0,5


In [28]:
# Merge medical_df with bert_embeddings_df using the uniqueID column
merged_df = pd.merge(medical_df, bert_embeddings_df, on='uniqueID', how='inner')

In [29]:
# Save the merged DataFrame to a CSV file
merged_df.to_csv("embedded_review.csv", index=False)

In [30]:
# Save the merged DataFrame to a CSV file in the specified directory
merged_df.to_csv(r'C:\Users\Bryson\OneDrive\Desktop\embedded_review\embedded_review.csv', index=False)

In [31]:
from google.colab import files

# Save the merged DataFrame to a CSV file
merged_df.to_csv("embedded_review.csv", index=False)

# Download the CSV file to your local machine
files.download("embedded_review.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>